# Tutorial 2

## CSC420 - Winter 2023

## Acknowledgment: Babak Taati
### Edited by: Shayan Shekarforoush

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
import cv2

First and second order derivates of images, namely gradients and laplacian, can be used to detect edges. For the discrete intensity function, we can approximate them using finite differences.

## Image Laplacian

In [ ]:
img = cv2.imread('./window.jpeg') # numpy array
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [ ]:
laplacian = cv2.Laplacian(gray, cv2.CV_64F)

In [ ]:
fig = plt.figure(figsize=(15,15))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

ax1.imshow(gray,cmap = 'gray')
ax1.title.set_text('Original'), ax1.set_xticks([]), ax1.set_yticks([])
ax2.imshow(laplacian,cmap = 'gray')
ax2.title.set_text('Laplacian'), ax2.set_xticks([]), ax2.set_yticks([]);

## Smooth first ...

In [ ]:
plt.figure(figsize=(7,7))
blur = cv2.GaussianBlur(gray, (5,5), 1)
plt.imshow(blur, cmap = 'gray')
plt.xticks([]), plt.yticks([]);

In [ ]:
laplacian = cv2.Laplacian(blur, cv2.CV_64F)

In [ ]:
fig = plt.figure(figsize=(15,15))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)
ax1.imshow(gray,cmap = 'gray')
ax1.title.set_text('Original'), ax1.set_xticks([]), ax1.set_yticks([])
ax2.imshow(laplacian,cmap = 'gray')
ax2.title.set_text('Laplacian'), ax2.set_xticks([]), ax2.set_yticks([]);

## Laplacian kernel

$$
\frac{\partial^2 f}{\partial x^2} \approx f(x+1) + f(x-1) - 2f(x) \Rightarrow k_x = [1, -2, 1] \\
\frac{\partial^2 f}{\partial y^2} \approx f(y+1) + f(y-1) - 2f(y) \Rightarrow k_y = [1, -2, 1]^T \\
k = k_x + k_y = \begin{bmatrix} 0 & 1 & 0 \\ 1 & -4 & 1 \\ 0 & 1 & 0 \end{bmatrix}
$$

In [ ]:
L = [[0,1,0],[1,-4,1],[0,1,0]]
L = np.asanyarray(L, np.float32)
print(L)

In [ ]:
dst = cv2.filter2D(blur, -1, kernel=L)
plt.figure(figsize=(7,7))
plt.imshow(dst, cmap = 'gray')
plt.xticks([]), plt.yticks([]);

## Is it the same?

In [ ]:
print([laplacian.min(), laplacian.max()])
print([dst.min(), dst.max()])

In [ ]:
whos

### Solution:  use ddepth=cv2.CV_64F

In [ ]:
dst = cv2.filter2D(blur, ddepth=cv2.CV_64F, kernel=L)

plt.figure(figsize=(7,7))
plt.imshow(dst, cmap = 'gray')
plt.xticks([]), plt.yticks([]);

In [ ]:
print([laplacian.min(), laplacian.max()])
print([dst.min(), dst.max()])

In [ ]:
np.allclose(laplacian, dst)

## Sobel filter

**Image Gradient:**
$$
\frac{\partial f}{\partial x} \approx f(x + 1) - f(x - 1) \Rightarrow k_x = [-1, 0, 1] \\
\frac{\partial f}{\partial y} \approx f(y + 1) - f(y - 1) \Rightarrow k_y = [-1, 0, 1]^T
$$
**Sobel filter:** Computing the gradients with (gaussian) smoothing.
$$
k_x = [1, 2, 1]^T * [-1, 0, 1] = \begin{bmatrix} -1 & 0 & 1 \\ -2 & 0 & 2 \\ -1 & 0 & 1 \end{bmatrix} \\
k_y = [-1, 0, 1]^T * [1, 2, 1] = \begin{bmatrix} -1 & -2 & -1 \\ 0 & 0 & 0 \\ 1 & 2 & 1 \end{bmatrix}
$$
This can be extended to larger kernels.

In [ ]:
sobelx = cv2.Sobel(blur, cv2.CV_64F, 1, 0, ksize=5)
sobely = cv2.Sobel(blur, cv2.CV_64F, 0, 1, ksize=5)

fig = plt.figure(figsize=(15,15))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

ax1.imshow(sobelx, cmap='gray')
ax1.title.set_text('Sobel X'), ax1.set_xticks([]), ax1.set_yticks([])
ax2.imshow(sobely, cmap='gray')
ax2.title.set_text('Sobel Y'), ax2.set_xticks([]), ax2.set_yticks([]);

**The light and dark regions correspond to positive and negativ gradients, respectively.**

**Caveat: If we replace cv2.CV_64F with -1, we miss negative slopes.**

In [ ]:
blended = cv2.addWeighted(src1=sobelx,alpha=0.5,src2=sobely,beta=0.5,gamma=0)

plt.figure(figsize=(7,7))
plt.imshow(blended, cmap = 'gray');

In [ ]:
# grad_mag = np.sqrt((sobelx**2) + (sobely**2)) # gradient magnitude
# grad_mag = abs(sobelx) + abs(sobely); # gradient first norm
grad_mag = cv2.addWeighted(src1=abs(sobelx), alpha=0.5, src2=abs(sobely), beta=0.5, gamma=0)

plt.figure(figsize=(7,7))
plt.imshow(grad_mag,cmap='gray')
plt.xticks([]), plt.yticks([]);

In [ ]:
print([sobelx.min(), sobelx.max()]) 
print([grad_mag.min(), grad_mag.max()])

### Histogram

In [ ]:
hist, bins = np.histogram(grad_mag, 3660, [0,3660])
plt.figure(figsize=(10,5))
plt.plot(hist)

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(hist)
plt.xlim([0,200])

In [ ]:
th = grad_mag > 1000
plt.figure(figsize=(7,7))
plt.imshow(th, cmap='gray')
plt.xticks([]), plt.yticks([]);

### Try different thresholds! Can you find a threshold that gets all the edges, but nothing else?

In [ ]:
ret, th1 = cv2.threshold(grad_mag, 1000, 255, cv2.THRESH_BINARY)
plt.figure(figsize=(7,7))
plt.imshow(th1, cmap='gray')
plt.xticks([]), plt.yticks([]);

## Canny edge detector

In [ ]:
edges = cv2.Canny(gray, threshold1=75, threshold2=100)

fig = plt.figure(figsize=(15,15))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

ax1.imshow(gray, cmap = 'gray')
ax1.title.set_text('Original Image'), ax1.set_xticks([]), ax1.set_yticks([])
ax2.imshow(edges,cmap = 'gray')
ax2.title.set_text('Edge Image'), ax2.set_xticks([]), ax2.set_yticks([])

You can find more examples in [Image Processing in OpenCV](https://docs.opencv.org/4.5.2/d2/d96/tutorial_py_table_of_contents_imgproc.html) Tutorials.

# Morphological Transformations

Let's go through the ***Morphological Transformations***  tutorial (under ***Image Processing in OpenCV***)


In [ ]:
img = np.zeros(shape=(480,640),dtype=np.int16)
cv2.putText(img,text='T E S T - O o', org=(100, 100), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=4)
cv2.putText(img,text='0 1 2 3 4 ', org=(150, 200), fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=2, color=(255, 255, 255), thickness=4)
cv2.putText(img,text='5 6 7 8 9', org=(150, 300), fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=2, color=(255, 255, 255), thickness=4)
ret, img = cv2.threshold(img, 120, 255, cv2.THRESH_BINARY)
plt.figure(figsize=(7,7))
plt.imshow(img, cmap='gray')

### Erosion

In [ ]:
kernel = np.ones((3,3), np.uint8)
erosion = cv2.erode(img, kernel, iterations=1)
plt.figure(figsize=(7,7))
plt.imshow(erosion, cmap='gray')

In [ ]:
kernel = np.ones((7,7), np.uint8)
dilation = cv2.dilate(img, kernel, iterations=1)
plt.figure(figsize=(7,7))
plt.imshow(dilation, cmap='gray')

**Convolution is a linear operator.**

**Are morphological transformations linear?**

Linear transformations is a mapping from a vector space (***V***) to another vector space (***W***),  **f: V → W**

For any two vectors ***u*** and ***v*** in ***V*** and any scalar *c*

*   ***f(u+v) = f(u) + f(v)***
*   ***f***(*c* ***u***) = *c* ***f(u)***

**Closing = Dilation followed by Erosion**

In [ ]:
dilation = cv2.dilate(img, kernel, iterations=1)
close = cv2.erode(dilation, kernel, iterations=1)
plt.figure(figsize=(7,7))
plt.imshow(close, cmap='gray')

In [ ]:
closing = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
plt.figure(figsize=(7,7))
plt.imshow(closing, cmap='gray')

**Opening = Erosion followed by Dilation**

In [ ]:
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
plt.figure(figsize=(7,7))
plt.imshow(opening, cmap='gray')

## Opening can get rid of 'salt' noise, why?

In [ ]:
# salt noise
noisy_img = img.copy() 
R = np.random.rand(img.shape[0], img.shape[1]) > 0.70
noisy_img[R] = 255
plt.figure(figsize=(7,7))
plt.imshow(noisy_img, cmap='gray')

In [ ]:
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(noisy_img, cv2.MORPH_OPEN, kernel)
plt.figure(figsize=(7,7))
plt.imshow(opening, cmap='gray')

## Closing can get rid of 'pepper' noise, why?

In [ ]:
# pepper noise
noisy_img = img.copy() 
R = np.random.rand(img.shape[0], img.shape[1]) > 0.60
noisy_img[R] = 0
plt.figure(figsize=(7,7))
plt.imshow(noisy_img, cmap='gray')

In [ ]:
kernel = np.ones((3,3),np.uint8)
closing = cv2.morphologyEx(noisy_img, cv2.MORPH_CLOSE, kernel)
plt.figure(figsize=(7,7))
plt.imshow(closing, cmap='gray')

**Morphological Gradient = difference between dilation and erosion of an image**

In [ ]:
gradient = cv2.morphologyEx(img, cv2.MORPH_GRADIENT, kernel)
plt.figure(figsize=(7,7))
plt.imshow(gradient, cmap='gray')

**Looks like ~ edge detection! (only for binary images)**

## Kernels of different shapes (elliptical, cross, ...)

In [ ]:
# try kernel shapes that are not square
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
kernel

In [ ]:
kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (5,5))
kernel

In [ ]:
opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
plt.figure(figsize=(7,7))
plt.imshow(opening, cmap='gray')